<a href="https://www.kaggle.com/code/ahmedsaied3122/used-car-price-in-egypt?scriptVersionId=143325110" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

| Attribute | Description |
|----------|----------|
|Brand	|The brand of the car
|Model	|The Model of the car
|Body	|The Body of the car
|Year	|The year or edition of the model
|Color	|The color the car
|Fuel	|The type of fuel used by the car
|Kilometers	|The range kilometers driven in the car by the previous owner(s) in KM
|Engine	|The displacement volume of the engine in cc
|Transmission	|The type of transmission used by the car
|Price	|The price of the used car
|Gov	|The Gov in which the car is being sold or is available for purchase

# Importing Libraries

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Reading Data

In [ ]:
df = pd.read_csv('/kaggle/input/used-car-price-in-egypt/Cars.csv')

# Exploring Data

In [ ]:
df.head()# Will give you first 5 records

In [ ]:
#gain insights of your data set and see general information about each feature and the total number of non-null values in each variable
df.info()

- Well, as a first observation, we can see that we're lucky, because we don't have missing values in some of the columns.

- unnamed0 ----> drop column

In [ ]:
#You can get idea that gives summary statstics for only intersting numerical features in your data set.
df.describe()

 * Year Range : 1970 - 2022
 * Price Range : 3 -  471

In [ ]:
df.drop('Unnamed: 0' , axis =1 , inplace = True )

# Data Cleaning

In [ ]:
df_clean = df.copy()

In [ ]:
df_clean.head()

## Check Missing Values

In [ ]:
df.isnull().sum()

### Check Duplicated

In [ ]:
df_clean.duplicated().sum()

In [ ]:
df_clean.drop_duplicates(inplace=True)

In [ ]:
df_clean.duplicated().sum()

# Feature Engineering


In [ ]:
df_clean.Kilometers.value_counts()

In [ ]:
def Get_kilometers(Num):
    if Num == "More than 200000":
        Kilo = Num.split(" ")[2]    # Handling the Text 'More than 200000' by exchange it to integer 2000000

    else :
        Kilo = Num.split(sep="to")[1]
        Kilo = Kilo.strip()        # Handling '140000 to 159999' by Spliting them and Choose the Max
    return Kilo 
   

In [ ]:
df_clean['Kilometers'] = df_clean.Kilometers.apply(Get_kilometers)
df_clean['Kilometers'] = df_clean['Kilometers'].astype(int) # Convert the Column to integer
df_clean.head()

In [ ]:
#kilometer_map = {'0 to 9999': 1, '10000 to 19999': 2, '100000 to 119999': 3, '20000 to 29999': 4, '30000 to 39999': 5,
              #   '40000 to 49999': 6, '50000 to 59999': 7, '60000 to 69999': 8, '70000 to 79999': 9,  '80000 to 89999': 10, 
                  # '90000 to 99999': 11,  '100000 to 119999': 12,  '120000 to 139999': 13,  '140000 to 159999': 14,
                 #  '160000 to 179999': 15,  '180000 to 199999': 16, 'More than 200000': 17
            #     }

In [ ]:
#df_clean.Kilometers = df_clean.Kilometers.map(kilometer_map)


In [ ]:
def Get_Engine(Num):
    if (len(Num) > 7) : # Spliting The data whose Length is more than 7 by spliting the range and remove 'CC' an Take Maximum
        Eng = Num.split(sep="-")[1]
        Eng = Eng.strip('CC')
        Eng = int(Eng)
   
    else:# Spliting The data whose Length is less than 7 by remove 'CC' an Take Maximum 
        Eng = Num.strip('CC')
        Eng = int(Eng)

    return Eng

In [ ]:
df_clean['Engine'] = df_clean.Engine.apply(Get_Engine)
df_clean.head()

In [ ]:
df_clean.describe()

In [ ]:
max_Year = df_clean.Year.max()
max_Year

In [ ]:
df_clean['Age_of_Car'] = (max_Year - df_clean['Year'])

In [ ]:
df_clean.drop('Year',axis=1 ,inplace=True)

In [ ]:
df_clean.head()

In [ ]:
df_clean.Price = df.Price.apply(lambda x : x * 1000 )           # Function to Multiply each Value by 1000 
df_clean

# Data Visualisation  

- The term Univariate Analysis refers to the analysis of only one variable. The aim is to analyse and find out patterns specific to a single variable

In [ ]:
cat_cols = df_clean.select_dtypes(include='object').columns.tolist()
cat_cols

In [ ]:
for col in cat_cols:
    print(df_clean[col].value_counts())
    print('------------------------------------------------------------')

In [ ]:
plt.figure(figsize = (50 , 100))
for i , col in enumerate (cat_cols  ,1) :
    plt.subplot((len(cat_cols)//1) , 1 , i )
    sns.histplot(data = df_clean , x = col)
    plt.xticks(fontsize=40)
    plt.yticks(fontsize=40)
    plt.xlabel(col, fontsize=80)
    

- brand --->  most popular is Htheyundai
- Model ---> Although the most common __brand is Hyundai__, the most popular model is the __128__    
- Body --->  most popular is Sedan 
- Color ---> most popular is White              
- Fuel ---> It is natural that Benzine is the most, because natural gas is one of the types that appeared recently 
- Transmission ---> With the continuous development in the world of cars, automatic cars are modern and help trainees to drive well, __while manual cars are the basis for driving education, so it is natural that they are the most popular.__
- Gov ---> Given that __Cairo__ is the capital of Egypt and the center of life services in it, as it has __the largest number__ of families among the population, it is natural that the governorates use the most cars.

In [ ]:
num_cols = df_clean.select_dtypes(exclude = 'O').columns.tolist()
num_cols

In [ ]:
for col in num_cols:
    print(df_clean[col].value_counts())
    print('------------------------------------------------------------')

In [ ]:
plt.figure(figsize = (50 , 100))
for i , col in enumerate (num_cols ,1) :
    plt.subplot((len(num_cols)//1) , 1 , i )
    sns.histplot(data = df_clean , x = col)
    plt.xticks(fontsize=40)
    plt.yticks(fontsize=40)
    plt.xlabel(col, fontsize=80)

In [ ]:
plt.figure(figsize = (20 , 40))
for i , col in enumerate (num_cols ,1) :
    plt.subplot((len(num_cols)//1) , 1 , i )
    sns.boxplot(data = df_clean , x = col)

## Bivariate Analysis

- __Q.1 Is there a significant relationship between a Brand and their likelihood to Price ?__

In [ ]:
brand_price = df_clean.groupby('Brand')['Price'].mean().sort_values(ascending=False)
fig = px.bar(brand_price , color = brand_price.index )
fig.show()

- Hyundai cars are the highest.

- __Q.2 Is there a significant relationship between a Model and their likelihood to price ?__

In [ ]:
model_price = df_clean.groupby('Model')['Price'].mean().sort_values(ascending=False)
fig = px.bar(model_price , color = model_price.index )
fig.show()

- Although the most-produced car in the dataset is the __128__, it represents the lowest average price and the __Tipo is the most-produced__.

- __Q.3 Is there a significant relationship between a Body and their likelihood to price ?__

In [ ]:
body_price = df_clean.groupby('Body')['Price'].mean().sort_values(ascending=False)
fig = px.bar(body_price , color = body_price.index )
fig.show()

- __Q.4 Is there a significant relationship between a Fuel and their likelihood to price ?__

In [ ]:
fuel_price = df_clean.groupby('Fuel')['Price'].mean().sort_values(ascending=False)
fig = px.bar(fuel_price , color = fuel_price.index )
fig.show()

- __Q.5 Is there a significant relationship between a Transmission and their likelihood to price ?__

In [ ]:
Transmission_price = df_clean.groupby('Transmission')['Price'].mean().sort_values(ascending=False)
fig = px.bar(Transmission_price , color = Transmission_price.index )
fig.show()

In [ ]:
num_cols

- __Q.6 Does the Kilometers of a customer have a significant impact on their likelihood to Price?__

In [ ]:
fig = px.scatter(df_clean , x = (df_clean['Kilometers']),y= (df_clean['Price']), trendline='ols')
fig.show()

- It is natural for the relationship to be inverse, because with an increase in car traffic, it leads to a decrease in their prices

- __Q.7 Does the Engine of a customer have a significant impact on their likelihood to Price?__

In [ ]:
fig = px.scatter(df_clean , x = (df_clean['Engine']),y= (df_clean['Price']), trendline='ols')
fig.show()

- We note that there is a direct relationship that when the Engine increases, the price of the car increases

- __Q.8 Does the Age_of_Car of a customer have a significant impact on their likelihood to Price?__

In [ ]:
fig = px.scatter(df_clean , x = (df_clean['Age_of_Car']),y= (df_clean['Price']), trendline='ols')
fig.show()

- We note that the relationship is inverse, because the age of the car has increased and it has become old, so its price is cheap

# Data Preprocessing

## Outlier

In [ ]:
def clean_dataset(df):
  df1 = df[df['Age_of_Car'] <= 30 ]
  return df1
df_clean = clean_dataset(df_clean)

In [ ]:
sns.boxplot(df_clean.Age_of_Car)

In [ ]:
categorical_cols = list(df_clean.select_dtypes(include=['object']).columns)
categorical_cols

In [ ]:
numerical_cols = list(df_clean.select_dtypes(include=['int64', 'float64']).columns)
numerical_cols.remove('Price')
numerical_cols

# Data Splitting

In [ ]:
# Data Splitting into features and target
X = df_clean.drop('Price', axis=1)
y = df_clean['Price']

# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Data Preprocessing
from category_encoders import BinaryEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[('binary', BinaryEncoder())])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, ['Kilometers', 'Engine', 'Age_of_Car']),
        ('cat', categorical_transformer, ['Brand', 'Model', 'Body', 'Color', 'Fuel', 'Transmission', 'Gov'])])

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Model Building

In [ ]:
# Model Building
from sklearn.linear_model import LinearRegression

# Linear Regression
lr = LinearRegression()
lr.fit(X_train_preprocessed, y_train)

# Model Evaluation
from sklearn.metrics import mean_squared_error, r2_score

# Predictions
y_pred = lr.predict(X_test_preprocessed)



# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))


In [ ]:
# Ridge Regression
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(X_train_preprocessed, y_train)

# Predictions
y_pred = ridge.predict(X_test_preprocessed)

# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
# Lasso Regression
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(X_train_preprocessed, y_train)

# Predictions
y_pred = lasso.predict(X_test_preprocessed)

# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
#polynomial regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

poly = make_pipeline(PolynomialFeatures(3), Ridge())
poly.fit(X_train_preprocessed, y_train)

# Predictions
y_pred = poly.predict(X_test_preprocessed)

# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
#DecisionTreeRegressor 
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
dt.fit(X_train_preprocessed, y_train)

# Predictions
y_pred = dt.predict(X_test_preprocessed)

# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
#RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor


rf = RandomForestRegressor(n_estimators=10 ,random_state=42, max_depth=19)
rf.fit(X_train_preprocessed, y_train)

# Predictions
y_pred = rf.predict(X_test_preprocessed)

# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
parameters = {
    'n_estimators':range(8,12),
    'max_depth': range(15,25),
}
clf = RandomForestRegressor()
RCV = RandomizedSearchCV(estimator=clf, 
                   param_distributions=parameters, 
                   n_iter=3,
                   cv=3)

In [ ]:
RCV.fit(X_train_preprocessed,y_train)

In [ ]:
RCV.best_params_

In [ ]:
#RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor


rf = RandomForestRegressor(n_estimators=11 ,random_state=42, max_depth=16)
rf.fit(X_train_preprocessed, y_train)

# Predictions
y_pred = rf.predict(X_test_preprocessed)

# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
rf.feature_importances_


In [ ]:
column_names = preprocessor.get_feature_names_out()

In [ ]:
column_names.shape

In [ ]:
 rf.feature_importances_.shape

In [ ]:
importance = rf.feature_importances_
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
pd.DataFrame({'Feature': column_names, 'Importance': rf.feature_importances_.round(3)}).sort_values('Importance', ascending=False)

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(x=rf.feature_importances_, y=column_names)

In [ ]:
# BaggingRegressor

from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

lin_reg = LinearRegression()
ridge = Ridge()
lasso = Lasso()
dt_reg = DecisionTreeRegressor(max_depth=10)

bag_reg = BaggingRegressor(dt_reg, n_estimators=100, bootstrap=True, random_state=42,
                           max_features=0.8)
                           

# Fitting the data
bag_reg.fit(X_train_preprocessed, y_train)

# Checking the score
print('Training Score: ', bag_reg.score(X_train_preprocessed, y_train))
print('Testing Score: ', bag_reg.score(X_test_preprocessed, y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
parameters = {
    'n_estimators':[100,200,300,400,500],
    'max_features': [0.7,0.8,0.9],
}
clf = BaggingRegressor()
RCV = RandomizedSearchCV(estimator=clf, 
                   param_distributions=parameters, 
                   n_iter=3,
                   cv=3)

In [ ]:
RCV.fit(X_train_preprocessed,y_train)

In [ ]:
RCV.best_params_

In [ ]:
# BaggingRegressor

from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

lin_reg = LinearRegression()
ridge = Ridge()
lasso = Lasso()
dt_reg = DecisionTreeRegressor(max_depth=10)

bag_reg = BaggingRegressor(dt_reg, n_estimators=100, bootstrap=True, random_state=42,
                           max_features=0.8)
                           

# Fitting the data
bag_reg.fit(X_train_preprocessed, y_train)

# Checking the score
print('Training Score: ', bag_reg.score(X_train_preprocessed, y_train))
print('Testing Score: ', bag_reg.score(X_test_preprocessed, y_test))

In [ ]:
# AdaBoost Regressor

from sklearn.ensemble import AdaBoostRegressor

ada_reg = AdaBoostRegressor(n_estimators=200, random_state=42)

# Fitting the data
ada_reg.fit(X_train_preprocessed, y_train)

# Checking the score
print('Training Score: ', ada_reg.score(X_train_preprocessed, y_train))
print('Testing Score: ', ada_reg.score(X_test_preprocessed, y_test))

In [ ]:
# GradientBoostingRegressor

from sklearn.ensemble import GradientBoostingRegressor

grad_reg = GradientBoostingRegressor(n_estimators=200, max_depth=3, random_state=42)

# Fitting the data
grad_reg.fit(X_train_preprocessed, y_train)

# Checking the score
print('Training Score: ', grad_reg.score(X_train_preprocessed, y_train))
print('Testing Score: ', grad_reg.score(X_test_preprocessed, y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
parameters = {
    'n_estimators':[100,200,300,400,500],
    'max_depth': range(2,8),
}
clf = GradientBoostingRegressor()
RCV = RandomizedSearchCV(estimator=clf, 
                   param_distributions=parameters, 
                   n_iter=3,
                   cv=3)

In [ ]:
RCV.fit(X_train_preprocessed,y_train)

In [ ]:
RCV.best_params_

In [ ]:
# GradientBoostingRegressor

from sklearn.ensemble import GradientBoostingRegressor

grad_reg = GradientBoostingRegressor(n_estimators=300, max_depth=5, random_state=42)

# Fitting the data
grad_reg.fit(X_train_preprocessed, y_train)

# Checking the score
print('Training Score: ', grad_reg.score(X_train_preprocessed, y_train))
print('Testing Score: ', grad_reg.score(X_test_preprocessed, y_test))

In [ ]:
pip install xgboost

In [ ]:
# XGBoost

from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=300, max_depth=3, random_state=42)

# Fitting the data
xgb.fit(X_train_preprocessed, y_train)

# Checking the score
print('Training Score: ', xgb.score(X_train_preprocessed, y_train))
print('Testing Score: ', xgb.score(X_test_preprocessed, y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
parameters = {
    'n_estimators':[100,200,300 ,400 ,500],
    'max_depth': range(3,10),
}
clf = XGBRegressor()
RCV = RandomizedSearchCV(estimator=clf, 
                   param_distributions=parameters, 
                   n_iter=3,
                   cv=3)

In [ ]:
RCV.fit(X_train_preprocessed,y_train)

In [ ]:
RCV.best_params_

In [ ]:
# XGBoost

from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=100, max_depth=5, random_state=42)

# Fitting the data
xgb.fit(X_train_preprocessed, y_train)

# Checking the score
print('Training Score: ', xgb.score(X_train_preprocessed, y_train))
print('Testing Score: ', xgb.score(X_test_preprocessed, y_test))

In [ ]:
Models_Summarization = pd.DataFrame(
                {
                    
'Training Accuracy': [round(lr.score(X_train_preprocessed, y_train) * 100, 2), 
                                          round(poly.score(X_train_preprocessed, y_train) * 100, 2), 
                                          round(dt.score(X_train_preprocessed, y_train) * 100, 2), 
                                          round(rf.score(X_train_preprocessed, y_train) * 100, 2), 
                                          round(bag_reg.score(X_train_preprocessed, y_train) * 100, 2),
                                          round(ada_reg.score(X_train_preprocessed, y_train) * 100, 2),
                                          round(grad_reg.score(X_train_preprocessed, y_train) * 100, 2),
                                          round(xgb.score(X_train_preprocessed, y_train) * 100, 2)], 
'Teasting Accuracy': [round(lr.score(X_test_preprocessed, y_test) * 100, 2), 
                                         round(poly.score(X_test_preprocessed, y_test) * 100, 2), 
                                         round(dt.score(X_test_preprocessed, y_test) * 100, 2), 
                                         round(rf.score(X_test_preprocessed, y_test) * 100, 2), 
                                         round(bag_reg.score(X_test_preprocessed, y_test) * 100, 2), 
                                         round(ada_reg.score(X_test_preprocessed, y_test) * 100, 2),
                                         round(grad_reg.score(X_test_preprocessed, y_test) * 100, 2),
                                         round(xgb.score(X_test_preprocessed, y_test) * 100, 2)], 
              
                },
    
    index = ['Linear Regression', 'Polynomial Regression', ' DecisionTreeRegressor', 
             'Random Forest Regressor', 'BaggingRegressor',
             'AdaBoost Regressor' , 'GradientBoostingRegressor','XGBoost']
            )

In [ ]:
Models_Summarization

# Saving 

In [ ]:
import pickle

In [ ]:
# Preprocessor Saving
import pickle
pickle.dump(preprocessor, open('preprocessor.pkl', 'wb'))

# Model Saving
pickle.dump(grad_reg, open('model.pkl', 'wb'))

In [ ]:
# Load Preprocessor
preprocessor = pickle.load(open('preprocessor.pkl', 'rb'))
model = pickle.load(open('model.pkl', 'rb'))


## <center>Thanks For Following </center>
